In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import tree
import tensorflow as tf
from tensorflow import keras
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
import joblib
from xgboost import XGBClassifier

In [3]:
# Load features and labels data
#NOTE: All variables say aapl but actually we are dealing with mixed data.

with open('mixed_one_gram_features_labelled_df.pkl', 'rb') as f:
    labelled_aapl_news_df = pickle.load(f)

In [4]:
trainX_df=labelled_aapl_news_df['features']
trainY_df=labelled_aapl_news_df['label']

trainX = np.stack(trainX_df)

trainY_df=trainY_df.astype('int')
trainY = np.stack(trainY_df)

print(trainX.shape)
print(trainY.shape)
trainX

(91857, 1039)
(91857,)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
#pca = PCA(n_components=700)
#pComponents= pca.fit_transform(trainX)
#sum([float(x) for x in pca.explained_variance_ratio_])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.1, random_state=23)

In [7]:
print(X_train.shape)
print(y_train.shape)

(82671, 1039)
(82671,)


In [9]:
print("training Logistic Regression now ...")

logistic_regression = LogisticRegression(max_iter=500, verbose=15, n_jobs=-1)
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)
acc = accuracy_score(y_test,y_pred)

print('Accuracy of Logistic Regression classifier is: %f ' % acc)

with open('logistic_regression-pca-one-gram-one-day-mixed-data-model.pkl', 'wb') as f:
    pickle.dump(logistic_regression, f)

training Logistic Regression now ...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  5.2min finished


Accuracy of Logistic Regression classifier is: 0.570433 


In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = confusion_matrix(y_test,y_pred, labels=[1,-1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[1,-1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred,labels=[1,-1])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[2175 2231]
 [1715 3065]]
Outcome values : 
 2175 2231 1715 3065
Classification report : 
               precision    recall  f1-score   support

           1       0.56      0.49      0.52      4406
          -1       0.58      0.64      0.61      4780

    accuracy                           0.57      9186
   macro avg       0.57      0.57      0.57      9186
weighted avg       0.57      0.57      0.57      9186



In [12]:
# KFold Cross Validation approach
kf = model_selection.KFold(n_splits=5, shuffle=False)
kf.split(trainX)
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []

# Iterate over each train-test split
for train_index, test_index in kf.split(trainX):
    print(train_index)
    print(test_index)
    # Split train-test
    X_split_train, X_split_test = trainX_df.iloc[train_index], trainX_df.iloc[test_index]
    y_split_train, y_split_test = trainY_df[train_index], trainY_df[test_index]
    
    X_split_train = np.stack(X_split_train)
    X_split_test = np.stack(X_split_test)
    y_split_train=y_split_train.astype('int')
    y_split_train = np.stack(y_split_train)
    y_split_test=y_split_test.astype('int')
    y_split_test = np.stack(y_split_test)
    
    # Train the model
    model = logistic_regression.fit(X_split_train, y_split_train)
    # Append to accuracy_model the accuracy of the model
    accuracy_model.append(accuracy_score(y_split_test, model.predict(X_split_test), normalize=True)*100)
        
print(accuracy_model)

[18372 18373 18374 ... 91854 91855 91856]
[    0     1     2 ... 18369 18370 18371]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.3min finished


[    0     1     2 ... 91854 91855 91856]
[18372 18373 18374 ... 36741 36742 36743]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.8min finished


[    0     1     2 ... 91854 91855 91856]
[36744 36745 36746 ... 55112 55113 55114]


...

[    0     1     2 ... 91854 91855 91856]
[55115 55116 55117 ... 73483 73484 73485]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.3min finished


[    0     1     2 ... 73483 73484 73485]
[73486 73487 73488 ... 91854 91855 91856]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.7min finished


[51.05595471369475, 50.631395602003046, 50.623264928419786, 50.677698546622395, 52.00043546894562]


In [10]:
print("training Random Forest classifier now ...")

random_forest_classifier = RandomForestClassifier(n_estimators=1000, n_jobs=-1, verbose=15)
random_forest_classifier.fit(X_train,y_train)
#increase n_estimators if you face issues in training
y_pred=random_forest_classifier.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print('Accuracy of Random Forest classifier is: %f ' % acc)

with open('random-forest-pca-one-gram-one-day-mixed-data-model.pkl', 'wb') as f:
    pickle.dump(random_forest_classifier, f)

training Random Forest classifier now ...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000

building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.4s


building tree 9 of 1000
building tree 10 of 1000

...

[Parallel(n_jobs=8)]: Done 211 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 212 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 213 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 214 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 215 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 216 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 217 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 218 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 219 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 220 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 221 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 222 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 223 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 224 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 225 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 226 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Do

[Parallel(n_jobs=8)]: Done 457 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 458 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 459 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 460 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 461 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 462 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 463 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 464 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 465 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 466 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 467 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 468 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 469 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 470 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 471 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 472 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Do

[Parallel(n_jobs=8)]: Done 704 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 705 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 706 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 707 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 708 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 709 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 710 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 711 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 712 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 713 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 714 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 715 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 716 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 717 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 718 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 719 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Do

[Parallel(n_jobs=8)]: Done 952 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 953 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 954 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 955 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 956 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 957 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 958 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 959 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 960 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 961 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 962 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 963 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 964 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 965 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 966 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 967 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Do

Accuracy of Random Forest classifier is: 0.739386 


In [11]:
print("training Random Forest XGBoost classifier now ...")

random_forest_xgBoost_classifier = XGBClassifier(n_estimators=3000, n_jobs=-1, verbosity=2, max_depth=30)
random_forest_xgBoost_classifier.fit(X_train,y_train)

y_pred=random_forest_xgBoost_classifier.predict(X_test)
predictions = [round(value) for value in y_pred]
acc=accuracy_score(y_test,predictions)
print('Accuracy of Random Forest XGBoost classifier is: %f ' % acc)

joblib.dump(random_forest_xgBoost_classifier, 'random_forest-XGBoost-pca-one-gram-one-day-mixed-data-3k-30-model.pkl')

training Random Forest XGBoost classifier now ...
[13:39:26] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 2362 extra nodes, 0 pruned nodes, max_depth=30
[13:39:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:39:28] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 3152 extra nodes, 0 pruned nodes, max_depth=30
[13:39:30] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 2802 extra nodes, 0 pruned nodes, max_depth=30
[13:39:32] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree prun

[13:41:00] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 420 extra nodes, 0 pruned nodes, max_depth=30
[13:41:02] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 400 extra nodes, 0 pruned nodes, max_depth=30
[13:41:04] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 756 extra nodes, 0 pruned nodes, max_depth=30
[13:41:06] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 576 extra nodes, 0 pruned nodes, max_depth=30
[13:41:08] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 370 extra nodes, 0 pruned nodes, max_depth=30
[13:41:10] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 318 extr

[13:42:40] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 394 extra nodes, 0 pruned nodes, max_depth=30
[13:42:42] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 218 extra nodes, 0 pruned nodes, max_depth=30
[13:42:44] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 742 extra nodes, 0 pruned nodes, max_depth=30
[13:42:46] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 556 extra nodes, 0 pruned nodes, max_depth=30
[13:42:48] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 384 extra nodes, 0 pruned nodes, max_depth=30
[13:42:50] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 582 extr

[13:44:19] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 378 extra nodes, 0 pruned nodes, max_depth=30
[13:44:22] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 768 extra nodes, 0 pruned nodes, max_depth=30
[13:44:24] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 214 extra nodes, 0 pruned nodes, max_depth=30
[13:44:26] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 1146 extra nodes, 0 pruned nodes, max_depth=30
[13:44:28] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 1744 extra nodes, 0 pruned nodes, max_depth=30
[13:44:30] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 422 ex

[13:46:00] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 390 extra nodes, 0 pruned nodes, max_depth=30
[13:46:02] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 212 extra nodes, 0 pruned nodes, max_depth=30
[13:46:04] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 582 extra nodes, 0 pruned nodes, max_depth=30
[13:46:06] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 160 extra nodes, 0 pruned nodes, max_depth=30
[13:46:08] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 412 extra nodes, 0 pruned nodes, max_depth=30
[13:46:10] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 670 extr

...

[15:22:01] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 216 extra nodes, 0 pruned nodes, max_depth=30
[15:22:03] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 334 extra nodes, 0 pruned nodes, max_depth=30
[15:22:05] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 292 extra nodes, 0 pruned nodes, max_depth=30
[15:22:08] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 414 extra nodes, 0 pruned nodes, max_depth=30
[15:22:10] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 252 extra nodes, 0 pruned nodes, max_depth=30
[15:22:12] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 206 extr

[15:23:43] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 210 extra nodes, 0 pruned nodes, max_depth=30
[15:23:45] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=30
[15:23:47] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 138 extra nodes, 0 pruned nodes, max_depth=30
[15:23:50] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 128 extra nodes, 0 pruned nodes, max_depth=30
[15:23:52] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 244 extra nodes, 0 pruned nodes, max_depth=30
[15:23:54] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 108 extr

[15:25:25] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 142 extra nodes, 0 pruned nodes, max_depth=30
[15:25:27] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 158 extra nodes, 0 pruned nodes, max_depth=30
[15:25:29] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 244 extra nodes, 0 pruned nodes, max_depth=30
[15:25:31] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 170 extra nodes, 0 pruned nodes, max_depth=30
[15:25:33] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 210 extra nodes, 0 pruned nodes, max_depth=30
[15:25:35] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 366 extr

[15:27:06] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 0 pruned nodes, max_depth=30
[15:27:08] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 0 pruned nodes, max_depth=30
[15:27:10] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 110 extra nodes, 0 pruned nodes, max_depth=30
[15:27:12] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=30
[15:27:15] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=30
[15:27:17] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 104 extra no

['random_forest-XGBoost-pca-one-gram-one-day-mixed-data-3k-30-model.pkl']

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = confusion_matrix(y_test,y_pred, labels=[1,-1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[1,-1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred,labels=[1,-1])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[3083 1323]
 [1075 3705]]
Outcome values : 
 3083 1323 1075 3705
Classification report : 
               precision    recall  f1-score   support

           1       0.74      0.70      0.72      4406
          -1       0.74      0.78      0.76      4780

    accuracy                           0.74      9186
   macro avg       0.74      0.74      0.74      9186
weighted avg       0.74      0.74      0.74      9186



In [17]:
# KFold Cross Validation approach
kf = model_selection.KFold(n_splits=5, shuffle=False)
kf.split(trainX)
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []

# Iterate over each train-test split
for train_index, test_index in kf.split(trainX):
    print(train_index)
    print(test_index)
    # Split train-test
    X_split_train, X_split_test = trainX_df.iloc[train_index], trainX_df.iloc[test_index]
    y_split_train, y_split_test = trainY_df[train_index], trainY_df[test_index]
    
    X_split_train = np.stack(X_split_train)
    X_split_test = np.stack(X_split_test)
    y_split_train=y_split_train.astype('int')
    y_split_train = np.stack(y_split_train)
    y_split_test=y_split_test.astype('int')
    y_split_test = np.stack(y_split_test)
    
    # Train the model
    model = random_forest_classifier.fit(X_split_train, y_split_train)
    # Append to accuracy_model the accuracy of the model
    accuracy_model.append(accuracy_score(y_split_test, model.predict(X_split_test), normalize=True)*100)
        
print(accuracy_model)

[18372 18373 18374 ... 91854 91855 91856]
[    0     1     2 ... 18369 18370 18371]


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200building tree 5 of 200

building tree 6 of 200building tree 7 of 200

building tree 8 of 200


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.4s


building tree 9 of 200
building tree 10 of 200


...

[Parallel(n_jobs=-1)]: Done 185 tasks      | elapsed:  1.8min


building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200
building tree 197 of 200
building tree 198 of 200
building tree 199 of 200
building tree 200 of 200


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  2.0min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    

[54.31090790333115, 53.04267363379055, 52.50666811822982, 51.74459746339339, 58.407272331391866]


[Parallel(n_jobs=8)]: Done 176 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 177 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 178 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 179 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 180 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 181 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 182 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 183 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 185 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.5s finished


In [11]:
print("current time:-", datetime.datetime.now())

#pca = PCA(n_components=700)
#pComponents= pca.fit_transform(trainX)
#print(sum([float(x) for x in pca.explained_variance_ratio_]))

#X_train, X_test, y_train, y_test = train_test_split(pComponents, trainY, test_size=0.1, random_state=23)
print(X_train.shape)
print(y_train.shape)

print("training SVM classifier now ...")

svm_classifier = svm.SVC(verbose=15).fit(X_train,y_train)
y_pred = svm_classifier.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Accuracy of SVM classifier is: %f ' % acc)

with open('svm-pca-one-gram-one-day-mixed-data-model.pkl', 'wb') as f:
    pickle.dump(svm_classifier, f)
print("current time:-", datetime.datetime.now())

current time:- 2021-04-19 00:14:11.608034
(82671, 1039)
(82671,)
training SVM classifier now ...
[LibSVM]Accuracy of SVM classifier is: 0.730459 
current time:- 2021-04-19 03:31:38.018000


In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix = confusion_matrix(y_test,y_pred, labels=[1,-1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(y_test,y_pred,labels=[1,-1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix = classification_report(y_test,y_pred,labels=[1,-1])
print('Classification report : \n',matrix)

Confusion matrix : 
 [[2979 1427]
 [1049 3731]]
Outcome values : 
 2979 1427 1049 3731
Classification report : 
               precision    recall  f1-score   support

           1       0.74      0.68      0.71      4406
          -1       0.72      0.78      0.75      4780

    accuracy                           0.73      9186
   macro avg       0.73      0.73      0.73      9186
weighted avg       0.73      0.73      0.73      9186



In [ ]:
# KFold Cross Validation approach
kf = model_selection.KFold(n_splits=5, shuffle=False)
kf.split(trainX)
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []

# Iterate over each train-test split
for train_index, test_index in kf.split(trainX):
    print(train_index)
    print(test_index)
    # Split train-test
    X_split_train, X_split_test = trainX_df.iloc[train_index], trainX_df.iloc[test_index]
    y_split_train, y_split_test = trainY_df[train_index], trainY_df[test_index]
    
    X_split_train = np.stack(X_split_train)
    X_split_test = np.stack(X_split_test)
    y_split_train=y_split_train.astype('int')
    y_split_train = np.stack(y_split_train)
    y_split_test=y_split_test.astype('int')
    y_split_test = np.stack(y_split_test)
    
    # Train the model
    model = svm_classifier.fit(X_split_train, y_split_train)
    # Append to accuracy_model the accuracy of the model
    accuracy_model.append(accuracy_score(y_split_test, model.predict(X_split_test), normalize=True)*100)
        
print(accuracy_model)

[18372 18373 18374 ... 91854 91855 91856]
[    0     1     2 ... 18369 18370 18371]
[LibSVM][    0     1     2 ... 91854 91855 91856]
[18372 18373 18374 ... 36741 36742 36743]
[LibSVM][    0     1     2 ... 91854 91855 91856]
[36744 36745 36746 ... 55112 55113 55114]
[LibSVM][    0     1     2 ... 91854 91855 91856]
[55115 55116 55117 ... 73483 73484 73485]
[LibSVM]